In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [9]:
train_df = pd.read_csv('/kaggle/input/mocktest/train (1).csv')
test_df = pd.read_csv('/kaggle/input/mocktest/test (1).csv')

In [10]:
train_df.head()

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,15658852.0,Chidimma,651.0,France,Female,35.0,4.0,132271.30,1.0,1.0,0.0,86587.37,0.0
1,1,15615176.0,Chukwubuikem,642.0,France,Male,35.0,7.0,0.00,2.0,1.0,1.0,83917.49,0.0
2,2,15771543.0,Macleod,662.0,France,Female,42.0,4.0,0.00,3.0,0.0,0.0,52337.97,1.0
3,3,15776824.0,Hs?,659.0,France,Female,38.0,5.0,121702.73,1.0,0.0,0.0,73564.44,0.0
4,4,15676937.0,Ts'ui,584.0,Spain,Male,47.0,7.0,0.00,2.0,1.0,1.0,86619.77,0.0


In [17]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               15000 non-null  int64  
 1   CustomerId       15000 non-null  float64
 2   Surname          15000 non-null  int64  
 3   CreditScore      15000 non-null  float64
 4   Geography        15000 non-null  int64  
 5   Gender           15000 non-null  int64  
 6   Age              15000 non-null  float64
 7   Tenure           15000 non-null  float64
 8   Balance          15000 non-null  float64
 9   NumOfProducts    15000 non-null  float64
 10  HasCrCard        15000 non-null  float64
 11  IsActiveMember   15000 non-null  float64
 12  EstimatedSalary  15000 non-null  float64
 13  Exited           15000 non-null  float64
dtypes: float64(10), int64(4)
memory usage: 1.6 MB


In [11]:
train_df.isnull().sum()

id                 0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

In [14]:
label_encoders = {}
for col in ['Geography', 'Gender', 'Surname']:
    le = LabelEncoder()
    combined = pd.concat([train_df[col].astype(str), test_df[col].astype(str)], axis=0)
    le.fit(combined)
    train_df[col] = le.transform(train_df[col].astype(str))
    test_df[col] = le.transform(test_df[col].astype(str))
    label_encoders[col] = le

In [15]:
X = train_df.drop(columns=['id', 'CustomerId', 'Exited'])
y = train_df['Exited']

In [16]:
X_train, X_val, y_train, y_val = train_test_split(
X, y, test_size=0.2, random_state=42, stratify=y
)

In [18]:
rf_model = RandomForestClassifier(
n_estimators=300,
max_depth=10,
min_samples_split=5,
min_samples_leaf=3,
random_state=42,
n_jobs=-1
)

In [19]:
rf_model.fit(X_train, y_train)

RandomForestClassifier(max_depth=10, min_samples_leaf=3, min_samples_split=5,
                       n_estimators=300, n_jobs=-1, random_state=42)

In [20]:
val_preds = rf_model.predict_proba(X_val)[:, 1]
roc_score = roc_auc_score(y_val, val_preds)
print(f'Validation ROC-AUC Score: {roc_score:.4f}')

Validation ROC-AUC Score: 0.9307


In [21]:
test_preds = rf_model.predict_proba(test_df.drop(columns=['id', 'CustomerId']))[:, 1]

In [22]:
submission = pd.DataFrame({
'id': test_df['id'],
'Exited': test_preds
})


submission.to_csv('random_forest_submission.csv', index=False)
print('Submission file saved as random_forest_submission.csv')

Submission file saved as random_forest_submission.csv
